# Setting up our Notebook

In [11]:
# Things to import

# Standard data, plotting, and mathematical tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# CNN
from sklearn.model_selection import train_test_split
from keras import utils
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Stats
import statsmodels.api as sm

# Importing the data

In [2]:
# Importing the data

dfs=['Non-Scaled TA Features 1H for BTC.csv']

# Determing the number of lags for our images

## Autoregressive Model Identification: The partial auto-correlation

The estimated partial auto-correlation function (PACF) can be used to identify the order of an autoregressive time series model. Values of $|\tau_h|$ greater or equal to $\frac{\Phi^{-1}(\alpha)}{\sqrt{T}}$, where $T$ is the number of observations and $\Phi(z)$ is the standard normal CDF, are significant lag $h$ partial autocorelations at the $\alpha$ confidence level.

We use the stattools package to estimate the PACF. The nlags parameter is the maximum number of lags used for PACF estimation.

In [5]:
# Defining a function to create PCA matrices

def PCA_creation_train_val_test(no_components, X_train, X_val, X_test):
    
    # Scaling the data with our X_train matrix
    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    
    # Fitting the PCA to our X_train matrix
    pca=PCA(n_components=no_components)
    X_train=pca.fit_transform(X_train)

    # Scaling the X_val and X_test 
    X_val=scaler.transform(X_val)
    X_test=scaler.transform(X_test)

    # Transforming the X_val and X_test
    X_val=pca.transform(X_val)
    X_test=pca.transform(X_test)
    
    return X_train, X_val, X_test

In [48]:
# Train and test splitting and scaling
X=pd.read_csv(dfs[0])
X=X.dropna()
y=X['Label']
X=X.drop('Label', axis=1)
X=X.drop('Unnamed: 0', axis=1)
X=X.drop('Percent Change', axis=1)

# Split into train and test
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.1, random_state=100, shuffle=False)
X_train, X_test, y_train, y_test=train_test_split(X_train, y_train, test_size=0.1, random_state=100, shuffle=False)


X_train, X_val, X_test=PCA_creation_train_val_test(20, X_train, X_val, X_test)

In [23]:
# Finding the best number of steps for our dataset

pacf = sm.tsa.stattools.pacf(X_train[:,0], nlags=8)
T = len(X_train[:,0])

sig_test = lambda tau_h: np.abs(tau_h) > 2.58/np.sqrt(T)

for i in range(len(pacf)):
    if sig_test(pacf[i]) == False:
        n_steps = i - 1
        print('n_steps set to', n_steps)
        break

n_steps set to 3


# Creating "images" of our sets

In [49]:
lags=3


# Image creation function
def image_creation(lags,X):
    X_images=[]
    
    for i in range(0, len(X)-lags):
        img=X[i,:]
        img=np.vstack((img, X[i+1,:]))
        img=np.vstack((img, X[i+2,:]))
        X_images.append(img)
    return X_images
    
# Getting the X matrices' images
X_train=image_creation(lags,X_train)
X_val=image_creation(lags,X_val)
X_test=image_creation(lags,X_test)
    
# Making sure our y vectors has the same length, now  
y_train=y_val[:-lags]
y_val=y_val[:-lags]
y_test=y_val[:-lags]

# Finalizing the shapes
num_classes = 2
input_shape = (lags, len(X.iloc[0]), 1)

# Add a dimension at the end to make sure images have shape (3, 20, 1)
X_train = np.expand_dims(X_train, -1)
X_val = np.expand_dims(X_val, -1)
X_test = np.expand_dims(X_test, -1)
print("X_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_val.shape[0], "val samples")
print(X_test.shape[0], "test samples")
# one-hot-encode the target variable
y_train = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

X_train shape: (30939, 3, 20, 1)
30939 train samples
3818 val samples
3435 test samples


# Grid Search

In [ ]:
# Function to create the model for Keras wrapper to scikit learn

# Now, we will run grid search on the optimizer. I NEED TO FILL IN THE PARAMETERS FROM THE GRID SEARCH!

def create_cnn_model(pool_type='max', conv_activation='sigmoid', dropout_rate=0.1, neurons=16, kern_size=2,
                    mid_layers=1, optimizer='adam', init_lr=1e-1, decay_steps=5000, decay_rates=0.1, ):
    # Create model
    model = keras.Sequential()
    
    # First layer: convolution
    model.add(keras.layers.Conv2D(neurons, kernel_size=(kern_size, kern_size), activation='relu', input_shape=input_shape)) 
        
    # First middle layer: convolution, pooling, and dropout
    if mid_layers==1:
        model.add(keras.layers.Conv2D(2*neurons, kernel_size=(kern_size,kern_size), activation=conv_activation))  
        if pool_type == 'max':
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        if pool_type == 'average':
            model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))
        if dropout_rate != 0:
            model.add(keras.layers.Dropout(rate=dropout_rate))     
    
    # Second middle layer: convolution, pooling, and dropout    
    if mid_layers==2:
        model.add(keras.layers.Conv2D(4*neurons, kernel_size=(kern_size, kern_size), activation=conv_activation))   
        if pool_type == 'max':
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        if pool_type == 'average':
            model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))
        if dropout_rate != 0:
            model.add(keras.layers.Dropout(rate=dropout_rate))  
        
    # Third middle layer: convolution, pooling, and dropout    
    if mid_layers==3:
        model.add(keras.layers.Conv2D(4*neurons, kernel_size=(kern_size, kern_size), activation=conv_activation))   
        if pool_type == 'max':
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        if pool_type == 'average':
            model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))
        if dropout_rate != 0:
            model.add(keras.layers.Dropout(rate=dropout_rate)) 
      
    # Penultimate layer with Flatten, Dense, and Dropout
    model.add(keras.layers.Flatten())         
    model.add(keras.layers.Dense(4*neurons, activation='sigmoid')) 
    # add a dropout layer if rate is not null    
    if dropout_rate != 0:
        model.add(keras.layers.Dropout(rate=dropout_rate)) 
    
    # Final layer with the softmax for classification
    model.add(keras.layers.Dense(3, activation='softmax'))
    
    # Running through the optimizers
    if optimizer=='adam':
        # Learning Rate Schedule
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rates)
    
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        
    if optimizer=='RMSprop':
        # Learning Rate Schedule
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rates)
    
        optimizer = keras.optimizers.RMSProp(learning_rate=lr_schedule)
        
    if optimizer=='SGD':
        # Learning Rate Schedule
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rates)
    
        optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
        
    # Compile model
    model.compile( 
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )    
    return model

In [ ]:
# Optimize the model with grid search

# Grid search parameters
n_epochs_cv = 20 # Number of epochs for our grid search
n_cv = 3 # Number of cross validations

# Create model to feed to our parameter grid search
model = KerasClassifier(build_fn=create_mlp_model_optimizer, verbose=1)

# Define parameters and values for grid search to check in our model
param_grid = {
    # The features to range over in our model
    'pool_type': ['max', 'average'],
    'conv_activation': ['sigmoid', 'tanh'],
    'neurons': [16,32],
    'dropout_rate':[0.1,0.2,0.3,0.4,0.5],
    'optimizer':['adam', 'RMSprop', 'SGD'],
    'kern_size':[2,3,4],
    'mid_layers':[0, 1, 2, 3],
     # The features to range over for our optimizer
    'init_lr':[1e-1,1e-2,1e-3,1e-4,1e-5],
    'decay_steps':range(1000,10000,1000),
    'decay_rates':[.5,.6,.7,.8,.9],
    # The number of epochs for each model
    'epochs': [n_epochs_cv],
}

# Creating the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=n_cv)

# Fitting the grid
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_params_)

In [ ]:
# Creating and compiling a model with the best parameters

cnn_model = create_cnn_model(pool_type = grid_result.best_params_['pool_type'],
                             conv_activation = grid_result.best_params_['conv_activation'],
                            neurons=grid_result.best_params_['neurons'],
                            dropout_rate=grid_result.best_params_['dropout_rate'],
                            optimizer=grid_result.best_params_['optimizer'],
                            kern_size=grid_result.best_params_['kern_size'],
                             init_lr=grid_result.best_params_['init_lr'],
                          decay_steps=grid_result.best_params_['decay_steps'],
                          decay_rates=grid_result.best_params_['decay_rates'])
# Fitting the model
history = cnn_model.fit(X_train, y_train, epochs=50, validation_data=(X_val,y_val), verbose=0)

In [ ]:
# Performance

history.history.keys()
pd.DataFrame(history.history).plot(figsize=(20, 16))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

y_pred=model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1) # Note, I need this for our accuracy and confusion matrix.


# Score
print(accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

In [ ]:
# Saving the model

cnn_model.save('Models/CNN 1H BTC.h5')

In [ ]:
# Reloading the model

cnn_model_BTC=model = keras.models.load_model('Models/CNN 1H BTC.h5')